# ⚽ Simulation Ligue des Champions – Modèle simple de prédiction

Ce notebook propose une simulation simplifiée de la Ligue des Champions, inspirée de ton projet.

**Objectifs :**
- Créer un dataset d'équipes avec un indice de performance
- Simuler des confrontations à élimination directe
- Estimer la probabilité pour chaque club d’atteindre les différentes phases
- Identifier les favoris et les outsiders

> 🔁 Les données sont simulées mais structurées comme un vrai cas d’étude.


## 1. Import des bibliothèques

In [ ]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

## 2. Création d'un dataset d'équipes

On crée une liste d’équipes avec :
- un coefficient de performance (inspiré de l'historique LDC),
- une confédération (pour varier),
- un statut de "top club" ou non.


In [ ]:
teams = [
    "Real Madrid", "FC Barcelona", "Bayern Munich", "Manchester City",
    "Liverpool", "Chelsea", "Paris Saint-Germain", "Juventus",
    "Atletico Madrid", "Borussia Dortmund", "Inter Milan", "AC Milan",
    "Arsenal", "Napoli", "RB Leipzig", "Tottenham"
]

# Indice de performance arbitraire (plus élevé = plus fort)
performance_index = [
    95, 92, 93, 94,
    90, 88, 91, 87,
    86, 85, 84, 83,
    82, 81, 80, 79
]

clubs = pd.DataFrame({
    "Team": teams,
    "PerfIndex": performance_index
})

clubs

## 3. Fonction de simulation de match

On suppose que :
- la probabilité de victoire dépend du rapport des indices de performance,
- un match se joue sur un résultat binaire (pas de match nul ici pour simplifier).

In [ ]:
def simulate_match(team_a, team_b, df_clubs):
    """Simule un match entre deux équipes selon leur indice de performance.

    Retourne le nom de l'équipe gagnante.
    """
    perf_a = df_clubs.loc[df_clubs['Team'] == team_a, 'PerfIndex'].values[0]
    perf_b = df_clubs.loc[df_clubs['Team'] == team_b, 'PerfIndex'].values[0]
    
    # Probabilité de victoire pour A
    p_a = perf_a / (perf_a + perf_b)
    
    if random.random() < p_a:
        return team_a
    else:
        return team_b

## 4. Simulation d'une phase à élimination directe

On peut simuler :
- les huitièmes de finale
- les quarts
- les demies
- la finale

On part ici de 16 équipes directement en phase finale.

In [ ]:
def simulate_round(teams_list, clubs_df, round_name="Round"):
    """Simule un tour (8e, quart, etc.) et renvoie les qualifiés."""
    random.shuffle(teams_list)
    winners = []
    print(f"\n=== {round_name} ===")
    for i in range(0, len(teams_list), 2):
        team_a = teams_list[i]
        team_b = teams_list[i+1]
        winner = simulate_match(team_a, team_b, clubs_df)
        print(f"{team_a} vs {team_b} → {winner}")
        winners.append(winner)
    return winners

In [ ]:
def simulate_knockout(clubs_df):
    """Simule une phase finale complète et renvoie le vainqueur."""
    teams_round = clubs_df['Team'].tolist()
    
    r16 = simulate_round(teams_round, clubs_df, "Huitièmes de finale")
    qf = simulate_round(r16, clubs_df, "Quarts de finale")
    sf = simulate_round(qf, clubs_df, "Demi-finales")
    final = simulate_round(sf, clubs_df, "Finale")
    
    winner = final[0]
    print(f"\n🏆 Vainqueur : {winner}\n")
    return winner

## 5. Exemple : une simulation complète

In [ ]:
winner_example = simulate_knockout(clubs)

## 6. Simulation Monte Carlo (1000 compétitions)

On répète la simulation plusieurs fois pour estimer :
- la probabilité pour chaque club de remporter la compétition.


In [ ]:
N_SIM = 1000
winners = []

for _ in range(N_SIM):
    winners.append(simulate_knockout(clubs))

winners_series = pd.Series(winners)
win_counts = winners_series.value_counts().reindex(clubs['Team']).fillna(0)
win_probs = win_counts / N_SIM

win_probs

### Visualisation des probabilités de victoire

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(x=win_probs.index, y=win_probs.values)
plt.xticks(rotation=45)
plt.ylabel("Probabilité de remporter la LDC")
plt.title("Probabilités de victoire (simulation Monte Carlo)")
plt.show()

## 7. Synthèse par club

In [ ]:
clubs_results = clubs.copy()
clubs_results['WinProb'] = win_probs.values
clubs_results.sort_values('WinProb', ascending=False)

## 8. Insights & Interprétation

- Les clubs possédant le plus fort indice de performance (ex : Real Madrid, Manchester City, Bayern) ont une probabilité de victoire nettement supérieure.
- La simulation montre l'effet de la variabilité : même un club moins bien classé peut parfois gagner.
- On peut enrichir ce modèle en ajoutant :
  - la forme récente,
  - des données de buts marqués/encaissés,
  - l'avantage domicile/extérieur,
  - des blessures ou suspensions.

Ce notebook sert de base à une démonstration de **simulation de tournoi** pour un poste de Marketing/Data Analyst.